In [4]:
# inputs 
"""
6 15
2 3 4 5 6   // it is for node-1, which has an edge with node-2, an edge with node-3, … 
1 3 4 5 6
1 2 4 5 6
1 2 3 5 6
1 2 3 4 6
1 2 3 4 5
"""
def load_data(filename):
    file = open(filename, 'r')
    
    file_list = list(file)
    n_vertex, n_edge = map(int, file_list[0].split()) 
    
    v_list = list(range(1, n_vertex+1))    
    
    e_list = []
    for row in file_list[1:]:
        v_edges = list(map(int, row.split()))
        e_list.append(v_edges)

    return v_list, e_list

In [13]:
class Vertex:
    # id, edges, partition_label
    def __init__(self, id, label="A"):
        self.id = id
        self.edges = []
        self.partition_label = label
    
    def __str__(self):
        e = ', '.join(list(map(str, self.edges)))
        return f"v({self.id}, e({e}), l({self.partition_label}))"
    
    def get_D_value(self):
        D_value = 0 # D = E - I
        # print(f"D_value for {self.id}")
        # print(', '.join(map(str, self.edges)))
        for edge in self.edges:
            if edge.left_id == self.id:
                other_v = edge.right_v
            elif edge.right_id == self.id:
                other_v = edge.left_v
            
            # print(other_v.partition_label)
            if other_v.partition_label != self.partition_label:
                D_value += 1 # external cost
            else:
                D_value -= 1 # internal cost
        
        return D_value
    
    def add_edge(self, edge):
        # undirected graph, ignore reverse direction
        for present_edge in self.edges:        
            if present_edge.left_id == edge.right_id and present_edge.right_id == edge.left_id:
                return
        
        self.edges.append(edge)
              
class Edge:
    # left_id, right_id, left_v, right_v
    def __init__(self, left_id:int, right_id:int, left_v:Vertex=None, right_v:Vertex=None):
        self.left_id = left_id
        self.right_id = right_id
        self.left_v = left_v
        self.right_v = right_v
    
    def __str__(self):
        return f"({self.left_id}, {self.right_id})"
    
class Graph:
    # vertices, edges
    def __init__(self, v_list, e_list):
        self.vertices = []
        self.edges = []
        
        # Create Vertex list
        for v_id in v_list:
            self.vertices.append(Vertex(v_id))
        
        # Create connection of edges and vertices
        for i in range(len(v_list)):
            left_id = v_list[i]
            
            # Retrieve vertex from self.vertices
            left_v = self.vertices[i]
            left_v_edges = e_list[i]
            
            # Create connection of left and right
            for right_id in left_v_edges:
                # row index = vertex index - 1
                right_v = self.vertices[right_id-1]
                
                # Create edge
                edge = Edge(left_id, right_id, left_v, right_v)
                self.edges.append(edge)
                # Add edge to left and right vertex
                left_v.add_edge(edge)
                right_v.add_edge(edge)
            
        # for v in self.vertices:
        #     print(v)
    
    def __str__(self):
        v = '\n'.join(map(str, self.vertices))
        
        return v
        
    def get_partition_cost(self):
        cost = 0
        
        for edge in self.edges:
            # print(f"left_l({edge.left_v.partition_label}), right_l({edge.right_v.partition_label})")
            if edge.left_v.partition_label != edge.right_v.partition_label:
                cost += 1
        
        return cost

In [91]:
class KernighanLin():
    def __init__(self, graph):
        self.graph = graph
    
    def partition(self):
        # initial partition: first half is group A, second half is B
        for i in range(int(len(self.graph.vertices)/2)):
            self.graph.vertices[i].partition_label = "A"
        for i in range(int(len(self.graph.vertices)/2), len(self.graph.vertices)):
            self.graph.vertices[i].partition_label = "B"
        print(self.graph)
        print("Initial partition cost: " + str(self.graph.get_partition_cost()))
        p = 0 # pass
        total_gain = 0
        
        len_v = len(self.graph.vertices)
        n_max_iter = int(len_v/2) + 2
       
        # repeat until g_max <= 0
        group_a = []
        group_b = []
    
        for i in range(len(self.graph.vertices)):
            if self.graph.vertices[i].partition_label == "A":
                group_a.append(self.graph.vertices[i])
            elif self.graph.vertices[i].partition_label == "B":
                group_b.append(self.graph.vertices[i])
    
        D_values = {v.id: v.get_D_value() for v in self.graph.vertices}
        gains = [] # [ ([a, b], gain), ... ]
    
        # while there are unvisited vertices
        for n_iter in range(int(len(self.graph.vertices)/2)): 
            # choose a pair that maximizes gain 
            max_gain = -1 * float("inf") # -infinity
            pair = []
            
            str_group_a = ', '.join(list(map(str, [v.id for v in group_a])))
            str_group_b = ', '.join(list(map(str, [v.id for v in group_b])))
            
            
            print(f"#{n_iter}, group_a({str_group_a}), group_b({str_group_b})")
            
        
            for a in group_a:
                for b in group_b:
                    c_ab = len(set(a.edges).intersection(b.edges))
                    gain = D_values[a.id] + D_values[b.id] - (2 * c_ab)
                    print(f"D({a.id}): {D_values[a.id]} + D({b.id}): {D_values[b.id]} - c_{a.id}{b.id}: {c_ab} = {gain}")                
                
                    if gain > max_gain:
                        max_gain = gain
                        pair = [a, b] 
        
            # mark that pair as visited
            a = pair[0]
            b = pair[1]
            group_a.remove(a)
            group_b.remove(b)
            gains.append([[a, b], max_gain])
            print(f"swap({a.id}, {b.id}), gain({max_gain})")
            # update D_values of other unvisited nodes connected to a and b, as if a and b are swapped
            for x in group_a:
                c_xa = len(set(x.edges).intersection(a.edges))
                c_xb = len(set(x.edges).intersection(b.edges))
                D_values[x.id] += 2 * (c_xa) - 2 * (c_xb)
        
            for y in group_b:
                c_yb = len(set(y.edges).intersection(b.edges))
                c_ya = len(set(y.edges).intersection(a.edges))
                D_values[y.id] += 2 * (c_yb) - 2 * (c_ya)
    
        # find j that maximizes the sum g_max
        g_max = -1 * float("inf")
        jmax = 0
        for j in range(1, len(gains) + 1):
            g_sum = 0
            for i in range(j):
                g_sum += gains[i][1]
        
            if g_sum > g_max:
                g_max = g_sum
                jmax = j
    
        if g_max > 0:
            # swap in graph
            for i in range(jmax):
                # find vertices and change their partition_label
                for v in self.graph.vertices:
                    if v.id == gains[i][0][0].id:
                        v.partition_label = "B"
                    elif v.id == gains[i][0][1].id:
                        v.partition_label = "A"
                
                    
                p += 1
                total_gain += g_max
                
                a_id = gains[i][0][0].id
                b_id = gains[i][0][1].id
                print(f"Pass: {p}, Swap: {a_id} & {b_id}, Gain: {g_max}")
            
        print("Total passes: " + str(p) + "\t\tTotal gain: " + str(total_gain) + "\t\tFinal partition cost: " + str(self.graph.get_partition_cost())) 
   

In [92]:
def main():
    filename = './ex_inputs.txt'
    v_list, e_list = load_data(filename)
    graph = Graph(v_list=v_list, e_list=e_list)
    kl = KernighanLin(graph)
    kl.partition()

In [93]:
main()

v(1, e((1, 2), (1, 3), (1, 4), (1, 5), (1, 6)), l(A))
v(2, e((1, 2), (2, 3), (2, 4), (2, 5), (2, 6)), l(A))
v(3, e((1, 3), (2, 3), (3, 4), (3, 5), (3, 6)), l(A))
v(4, e((1, 4), (2, 4), (3, 4), (4, 5), (4, 6)), l(B))
v(5, e((1, 5), (2, 5), (3, 5), (4, 5), (5, 6)), l(B))
v(6, e((1, 6), (2, 6), (3, 6), (4, 6), (5, 6)), l(B))
Initial partition cost: 18
#0, group_a(1, 2, 3), group_b(4, 5, 6)
D(1): 1 + D(4): 1 - c_14: 1 = 0
D(1): 1 + D(5): 1 - c_15: 1 = 0
D(1): 1 + D(6): 1 - c_16: 1 = 0
D(2): 1 + D(4): 1 - c_24: 1 = 0
D(2): 1 + D(5): 1 - c_25: 1 = 0
D(2): 1 + D(6): 1 - c_26: 1 = 0
D(3): 1 + D(4): 1 - c_34: 1 = 0
D(3): 1 + D(5): 1 - c_35: 1 = 0
D(3): 1 + D(6): 1 - c_36: 1 = 0
swap(1, 4), gain(0)
#1, group_a(2, 3), group_b(5, 6)
D(2): 1 + D(5): 1 - c_25: 1 = 0
D(2): 1 + D(6): 1 - c_26: 1 = 0
D(3): 1 + D(5): 1 - c_35: 1 = 0
D(3): 1 + D(6): 1 - c_36: 1 = 0
swap(2, 5), gain(0)
#2, group_a(3), group_b(6)
D(3): 1 + D(6): 1 - c_36: 1 = 0
swap(3, 6), gain(0)
Total passes: 0		Total gain: 0		Final part